In [1]:
import RPi.GPIO as GPIO
import time
from ADCDevice import *
import threading

In [2]:
import pyo

In [3]:
# start the audio server
s = pyo.Server(audio='jack').boot()
s.start()

Pyo warning: Buffer size set to Jack engine buffer size: 1024.


In [4]:
Z_Pin = 12      # define Z_Pin
adc = ADCDevice() # Define an ADCDevice class object

In [5]:
def setup():
    global adc
    if(adc.detectI2C(0x48)): # Detect the pcf8591.
        adc = PCF8591()
    elif(adc.detectI2C(0x4b)): # Detect the ads7830
        adc = ADS7830()
    else:
        print("No correct I2C address found, \n"
        "Please use command 'i2cdetect -y 1' to check the I2C address! \n"
        "Program Exit. \n");
        exit(-1)
    GPIO.setmode(GPIO.BOARD)        
    GPIO.setup(Z_Pin,GPIO.IN,GPIO.PUD_UP)   # set Z_Pin to pull-up mode

In [6]:
def destroy():
    adc.close()
    GPIO.cleanup()

In [7]:
def loop(sine):
    global stop
    val_X0 = 0
    val_Y0 = 0
    val_Z0 = 0
    while True:     
        val_Z = GPIO.input(Z_Pin)       # read digital value of axis Z
        val_Y = adc.analogRead(0)           # read analog value of axis X and Y
        val_X = adc.analogRead(1)
        sine.setFreq(312+val_X) 
        sine.setMul(val_Y/255*0.4+0.2)
        time.sleep(0.1)
        if stop:
            break  

In [8]:
setup()

Not found device in address 0x48
Found device in address 0x4b


In [9]:
sine = pyo.Sine(freq=312,mul=0.5).out(0,1)

In [10]:
stop = False
threading.Thread(target=loop,args=(sine)).start()

In [12]:
stop=True
sine.stop()

< Instance of Sine class >

In [ ]:
destroy()